In [ ]:
!mkdir mult_classification_sample
%cd ./mult_classification_sample

In [ ]:
%pip install transformers==4.47.1 fugashi==1.4.0 ipadic==1.0.0 pytorch-lightning==1.8.6 unidic-lite==1.0.8

In [10]:
import random
import glob
import json
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = 'tohoku-nlp/bert-base-japanese-whole-word-masking'

In [11]:
class BertForSequenceClassificationMultiLabel(torch.nn.Module):

    def __init__(self, model_name, num_labels):
        # model_name: Transformaers のモデル名
        # num_labels: ラベルの数

        super().__init__()

        # BERT のロード
        self.bert = BertModel.from_pretrained(model_name)

        # 線形変換を初期化
        self.liner = torch.nn.Linear(
            self.bert.config.hidden_size, num_labels
        )
    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        # データを入力し、BERT の最終層の出力を得る
        bert_output = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids
        )
        last_hidden_state = bert_output.last_hidden_state

        # [PAD] 以外のトークンで隠れ状態の平均をとる
        average_hidden_state = \
            (last_hidden_state * attention_mask.unsqueeze(-1)).sum(1) \
            / attention_mask.sum(1, keepdim=True)

        # 線形変換
        scores = self.liner(average_hidden_state)

        # 出力の形式を整える
        output = { 'logits': scores }

        # labels が入力に含まれていたら、損失を計算し、出力する
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float())
            output['loss'] = loss
        
        # 属性でアクセスできるようにする
        output = type ('bert_output', (object,), output)

        return output

In [ ]:
# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# ネガ/ポジ の2クラスの複数クラス分類を想定
bert_scml = BertForSequenceClassificationMultiLabel(MODEL_NAME, num_labels=2)
bert_scml = bert_scml.cuda()

In [ ]:
# chABSA-dataset のダウンロード
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
# zipを展開
!unzip chABSA-dataset.zip

In [ ]:
category_id = { 'negative': 0, 'neutral': 1, 'positive': 2  }

dataset = []
for file in glob.glob('chABSA-dataset/*.json'):
    data = json.load(open(file))
    # 各データから文章(text)を抜き出し、ラベル('labels')を作成
    for sentence in data['sentences']:
        text = sentence['sentence']
        labels = [0, 0, 0]
        for opinion in sentence['opinions']:
            labels[category_id[opinion['polarity']]] = 1
        sample = { 'text': text, 'labels': labels }
        dataset.append(sample)

In [ ]:
# 各データの形式を整える
max_length = 128
dataset_for_loader = []
for sample in dataset:
    text = sample['text']
    labels = sample['labels']
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True
    )
    encoding['labels'] = labels
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

# データセットの分割
random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]        # 学習用データ
dataset_val = dataset_for_loader[n_train:n_val]     # 検証用データ
dataset_test = dataset_for_loader[n_train+n_val:]   # テストデータ

# データセットからデータローダーを作成
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)


In [ ]:
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name, num_labels=num_labels
        )
    
    def training_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        loss = output.loss
        self.log('val_loss', loss)
    
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_scml(**batch)
        scores = output.logits
        labels_predicted = ( scores > 0 ).int()
        num_correct = ( labels_predicted == labels ).all(-1).sum().item()
        accuracy = num_correct / scores.size(0)
        self.log('accuracy', accuracy)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks = [checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME,
    num_labels=3,
    lr=1e-5
)
trainer.fit(model, dataloader_train, dataloader_val)
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')